# 多层索引
## 创建多层索引  MultiIndex

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, MultiIndex

- 以数组的方式创建多级行索引

MultiIndex.from_arrays([[一级索引], [二级索引]])

注意：一级索引的数量和二级索引的数量保持一致

In [2]:
index = MultiIndex.from_arrays([['张大妈', '张大妈', '张大妈', '李大爷','李大爷'],
                                ['周一', '周二', '周三', '周一', '周四']])


- 以元组方式创建多级索引

MultiIndex.from_tuples([(一级,二级), (), ])

In [11]:
index2 = MultiIndex.from_tuples([('张大妈', '周一'),
                                ('张大妈', '周二'),
                                ('张大妈', '周三'),
                                ('李大爷', '周一'),
                                ('李大爷', '周三')])

- product方式创建多级索引 【建议】

MultiIndex.from_product([[一级索引], [二级索引]])

生成行数 = [一级索引]的大小 * [二级索引]的大小

每一个"一级索引"，都有相同的"二级索引"

In [15]:
index3 = MultiIndex.from_product([['张大妈', '李大爷'],
                                  ['周一', '周二', '周三']])

In [16]:
df = DataFrame(np.random.randint(1,5, size=(6, 2)), 
               index=index3, columns=('白菜', '萝卜'))
df

白菜  萝卜
张大妈 周一   3   3
    周二   2   2
    周三   1   3
李大爷 周一   4   2
    周二   4   2
    周三   1   2

练习： 手机销售表

2017和2018两年的各季度的不同地区的不同手机销售情况

手机品牌：(Vivo, 华为, Iphone, Oppo)

地区： 西安，北京

In [39]:
index = MultiIndex.from_product([['西安', '北京'],
                               ['Vivo', '华为', 'Iphone', 'Oppo']])
columns = MultiIndex.from_product([['2017', '2018'],
                                   [1, 2, 3, 4]])
data = np.random.randint(100, 2000, size=(8, 8))

df = DataFrame(data ,index, columns)
df

2017                    2018                  
              1     2     3     4     1     2     3     4
西安 Vivo     197  1447  1736  1859   708   367   379  1080
   华为       898  1021  1668   196  1256  1674   837   986
   Iphone  1552   228   448   239  1232   794   584   936
   Oppo     991   722  1599  1062   649  1887   436  1527
北京 Vivo    1806  1939  1787  1135   258   701  1531  1325
   华为      1880   124  1007   311  1949  1157  1863   915
   Iphone   489   115  1022   511   275   851  1165   111
   Oppo    1036   698   167  1961   902   505  1017   341

# 多层索引操作
- df[(一级列， 二级列)]
- .loc[[行]， [列]]
- .iloc[[行号], [列号]]

注： 不能直接对二级索引进行操作，可以使用 (一级索引， 二级索引) 方式

In [40]:
# 查看2017年全年的华为手机的各地的销量
df.loc[[('西安', '华为'), ('北京', '华为')]]['2017']

,,1,2,3,4
西安,华为,898,1021,1668,196
北京,华为,1880,124,1007,311


In [36]:
df.iloc[[1, 5]]['2017']  # 直接使用索引值

,,1,2,3,4
西安,华为,653,832,500,713
北京,华为,1230,1284,429,1946


In [52]:
df.loc[['北京'], ['2017']]  # 查看北京地区的2017的销售情况

2017                  
              1     2     3     4
北京 Vivo    1806  1939  1787  1135
   华为      1880   124  1007   311
   Iphone   489   115  1022   511
   Oppo    1036   698   167  1961

In [56]:
# df.loc[:, [('2018', 3),('2018', 4)]]
df.loc[:,'2018'][[1, 2]] #  查看2018年第1和2季度的情况

1     2
西安 Vivo     708   367
   华为      1256  1674
   Iphone  1232   794
   Oppo     649  1887
北京 Vivo     258   701
   华为      1949  1157
   Iphone   275   851
   Oppo     902   505

In [57]:
s = df[('2018', 1)]
s  # Series

西安  Vivo       708
    华为        1256
    Iphone    1232
    Oppo       649
北京  Vivo       258
    华为        1949
    Iphone     275
    Oppo       902
Name: (2018, 1), dtype: int64

In [58]:
s.loc['西安']

Vivo       708
华为        1256
Iphone    1232
Oppo       649
Name: (2018, 1), dtype: int64

In [59]:
s.loc[[('西安', 'Oppo'), ('北京', 'Oppo')]]

西安  Oppo    649
北京  Oppo    902
Name: (2018, 1), dtype: int64

In [61]:
s.iloc[[3, 7]]

西安  Oppo    649
北京  Oppo    902
Name: (2018, 1), dtype: int64

In [62]:
# 创建3个以上的多级索引 -- Series
index_s = MultiIndex.from_product([['手机','平板'],
                                   ['西安','北京'],
                                   ['Vivo', 'Oppo', '华为']])
ss = Series(np.random.randint(100,1000, size=12),
           index=index_s)
ss

手机  西安  Vivo    928
        Oppo    808
        华为      762
    北京  Vivo    543
        Oppo    107
        华为      870
平板  西安  Vivo    680
        Oppo    411
        华为      468
    北京  Vivo    788
        Oppo    987
        华为      582
dtype: int64

In [64]:
ss.loc[('手机', '西安', 'Oppo')]

808

# 索引的堆操作
- stack()  将列索引转成行索引
- unstack() 将行索引转成列索引

level参数： 
- 默认为-1，即为最小级别（最后级别）
- 0 表示 第一级, 1 表示 第二有，依次类推

In [66]:
df.stack()  # 默认情况，将第二级索引转成行索引（最小级别-内）

2017  2018
西安 Vivo   1   197   708
          2  1447   367
          3  1736   379
          4  1859  1080
   华为     1   898  1256
          2  1021  1674
          3  1668   837
          4   196   986
   Iphone 1  1552  1232
          2   228   794
          3   448   584
          4   239   936
   Oppo   1   991   649
          2   722  1887
          3  1599   436
          4  1062  1527
北京 Vivo   1  1806   258
          2  1939   701
          3  1787  1531
          4  1135  1325
   华为     1  1880  1949
          2   124  1157
          3  1007  1863
          4   311   915
   Iphone 1   489   275
          2   115   851
          3  1022  1165
          4   511   111
   Oppo   1  1036   902
          2   698   505
          3   167  1017
          4  1961   341

In [67]:
df.stack(level=0)  # 通过level指定第几级列级别转成行索引

1     2     3     4
西安 Vivo   2017   197  1447  1736  1859
          2018   708   367   379  1080
   华为     2017   898  1021  1668   196
          2018  1256  1674   837   986
   Iphone 2017  1552   228   448   239
          2018  1232   794   584   936
   Oppo   2017   991   722  1599  1062
          2018   649  1887   436  1527
北京 Vivo   2017  1806  1939  1787  1135
          2018   258   701  1531  1325
   华为     2017  1880   124  1007   311
          2018  1949  1157  1863   915
   Iphone 2017   489   115  1022   511
          2018   275   851  1165   111
   Oppo   2017  1036   698   167  1961
          2018   902   505  1017   341

In [68]:
df.unstack()  # 将 最内的行索引转成 最内的列级别标签

2017                                                        ...  2018  \
        1                        2                       3       ...     2   
   Iphone  Oppo  Vivo    华为 Iphone Oppo  Vivo    华为 Iphone  Oppo ...  Vivo   
北京    489  1036  1806  1880    115  698  1939   124   1022   167 ...   701   
西安   1552   991   197   898    228  722  1447  1021    448  1599 ...   367   

                                                           
              3                        4                   
      华为 Iphone  Oppo  Vivo    华为 Iphone  Oppo  Vivo   华为  
北京  1157   1165  1017  1531  1863    111   341  1325  915  
西安  1674    584   436   379   837    936  1527  1080  986  

[2 rows x 32 columns]

In [70]:
df.unstack(level=0).unstack()  # 返回一个Series

2017  1  北京  Iphone     489
             Oppo      1036
             Vivo      1806
             华为        1880
         西安  Iphone    1552
             Oppo       991
             Vivo       197
             华为         898
      2  北京  Iphone     115
             Oppo       698
             Vivo      1939
             华为         124
         西安  Iphone     228
             Oppo       722
             Vivo      1447
             华为        1021
      3  北京  Iphone    1022
             Oppo       167
             Vivo      1787
             华为        1007
         西安  Iphone     448
             Oppo      1599
             Vivo      1736
             华为        1668
      4  北京  Iphone     511
             Oppo      1961
             Vivo      1135
             华为         311
         西安  Iphone     239
             Oppo      1062
                       ... 
2018  1  北京  Vivo       258
             华为        1949
         西安  Iphone    1232
             Oppo       649
             Vivo   

In [74]:
df.stack().unstack(level=1)

2017                     2018                  
     Iphone  Oppo  Vivo    华为 Iphone  Oppo  Vivo    华为
北京 1    489  1036  1806  1880    275   902   258  1949
   2    115   698  1939   124    851   505   701  1157
   3   1022   167  1787  1007   1165  1017  1531  1863
   4    511  1961  1135   311    111   341  1325   915
西安 1   1552   991   197   898   1232   649   708  1256
   2    228   722  1447  1021    794  1887   367  1674
   3    448  1599  1736  1668    584   436   379   837
   4    239  1062  1859   196    936  1527  1080   986

In [90]:
df.unstack(level=1).stack(level=0).stack(level=0).unstack(level=0).stack(level=0).unstack()

北京                       西安                  
       Iphone  Oppo  Vivo    华为 Iphone  Oppo  Vivo    华为
2017 1    489  1036  1806  1880   1552   991   197   898
     2    115   698  1939   124    228   722  1447  1021
     3   1022   167  1787  1007    448  1599  1736  1668
     4    511  1961  1135   311    239  1062  1859   196
2018 1    275   902   258  1949   1232   649   708  1256
     2    851   505   701  1157    794  1887   367  1674
     3   1165  1017  1531  1863    584   436   379   837
     4    111   341  1325   915    936  1527  1080   986

# 聚合操作
- mean()/max()/min()/std()/sum()

level指定行索引或列的等级

In [98]:
df

2017                    2018                  
              1     2     3     4     1     2     3     4
西安 Vivo     197  1447  1736  1859   708   367   379  1080
   华为       898  1021  1668   196  1256  1674   837   986
   Iphone  1552   228   448   239  1232   794   584   936
   Oppo     991   722  1599  1062   649  1887   436  1527
北京 Vivo    1806  1939  1787  1135   258   701  1531  1325
   华为      1880   124  1007   311  1949  1157  1863   915
   Iphone   489   115  1022   511   275   851  1165   111
   Oppo    1036   698   167  1961   902   505  1017   341

In [100]:
# level 为0 表示最外层的索引级别，外层的个数即为聚合的行数
df.max(axis=0,level=1)  # 行之间的最大值， level指定哪个行级别

2017                    2018                  
           1     2     3     4     1     2     3     4
Iphone  1552   228  1022   511  1232   851  1165   936
Oppo    1036   722  1599  1961   902  1887  1017  1527
Vivo    1806  1939  1787  1859   708   701  1531  1325
华为      1880  1021  1668   311  1949  1674  1863   986

In [101]:
df.max(axis=1)  # 列之间的最大值, 返回Series

西安  Vivo      1859
    华为        1674
    Iphone    1552
    Oppo      1887
北京  Vivo      1939
    华为        1949
    Iphone    1165
    Oppo      1961
dtype: int64

In [102]:
df.max(axis=1, level=0)  # 按列的最外级别，计算出每个列一级的最大值

2017  2018
西安 Vivo    1859  1080
   华为      1668  1674
   Iphone  1552  1232
   Oppo    1599  1887
北京 Vivo    1939  1531
   华为      1880  1949
   Iphone  1022  1165
   Oppo    1961  1017

In [103]:
df.max(axis=1, level=1)   # 找出两个年度的同一季度的最大销量

1     2     3     4
西安 Vivo     708  1447  1736  1859
   华为      1256  1674  1668   986
   Iphone  1552   794   584   936
   Oppo     991  1887  1599  1527
北京 Vivo    1806  1939  1787  1325
   华为      1949  1157  1863   915
   Iphone   489   851  1165   511
   Oppo    1036   698  1017  1961

In [107]:
# 计算出2017年Vivo手机的总销量
df.sum(axis=0, level=1).sum(axis=1,level=0).loc['Vivo', '2017']

11906

In [113]:
# 计算出各个地区的2018年度的平均销量, 保留2位小数
np.round(df.mean(axis=1, level=0).mean(axis=0, level=0)['2018'], 2)

北京    929.12
西安    958.25
Name: 2018, dtype: float64

In [130]:
df.iloc[:2]

2017                    2018                 
           1     2     3     4     1     2    3     4
西安 Vivo  197  1447  1736  1859   708   367  379  1080
   华为    898  1021  1668   196  1256  1674  837   986

In [128]:
np.max(df.iloc[:2])

2017  1     898
      2    1447
      3    1736
      4    1859
2018  1    1256
      2    1674
      3     837
      4    1080
dtype: int64